In [10]:
# load Facenet512 from deepface and fine tune it on our dataset
!pip install deepface
from deepface import DeepFace


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=795d7c20bdc3a13a2edf3f6fd1dfb466e506558f00e31bab6662e15bcd285556
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [8]:
import os
import gdown
from deepface.basemodels import Facenet
from deepface.commons import functions


def loadModel(
    url="https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5",
):

    model = Facenet.InceptionResNetV2(dimension=512)

    # -------------------------

    home = functions.get_deepface_home()

    if os.path.isfile(home + "/.deepface/weights/facenet512_weights.h5") != True:
        print("facenet512_weights.h5 will be downloaded...")

        output = home + "/.deepface/weights/facenet512_weights.h5"
        gdown.download(url, output, quiet=False)

    # -------------------------

    model.load_weights(home + "/.deepface/weights/facenet512_weights.h5")

    # -------------------------

    return model
Face512 = loadModel()

In [15]:
train_dir = "temp_faces"
test_dir = "temp_testfaces"

# -------------------------

list_train = []
for path in os.listdir(train_dir):
    list_train.append(train_dir + "/" + path)

list_test = []
for path in os.listdir(test_dir):
    list_test.append(test_dir + "/" + path)



In [16]:
class_names = []
for file_name in list_train:
    # print(file_name)
    # break
    class_name = file_name.split('/')[1].split('_')[0]
    # print(class_name)
    if class_name not in class_names:
        class_names.append(class_name)

# Get the number of classes
num_classes = len(class_names)
num_classes

1301

In [17]:
# Get the class names from list_train
class_names_train = []
for file_name in list_train:
    class_name = file_name.split('/')[1].split('_')[0]
    if class_name not in class_names_train:
        class_names_train.append(class_name)

# Get the class names from list_test
class_names_test = []
for file_name in list_test:
    class_name = file_name.split('/')[1].split('_')[0]
    if class_name not in class_names_test:
        class_names_test.append(class_name)

# Remove the classes that are in list_test but not in list_train
for class_name in class_names_test:
    if class_name not in class_names_train:
        list_test = [file_name for file_name in list_test if not file_name.startswith(f'{class_name}_')]


In [ ]:
list_train[0]

'temp_faces/0_0.jpg'

In [19]:
import cv2
import numpy as np

def load_images(file_list):
    images = []
    labels = []
    for file_name in file_list:
        class_name, label = file_name.split('/')[1].split('_')
        # img_path = os.path.join("", file_name)
        img_path = file_name
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (160, 160))
        images.append(img)
        labels.append(int(class_name))
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

train_images, train_labels = load_images(list_train)
test_images, test_labels = load_images(list_test)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import numpy as np

# Assuming your training labels are stored in a NumPy array called train_labels
num_classes = len(np.unique(train_labels))
print("Number of unique labels:", num_classes)

Number of unique labels: 1301


In [24]:
from sklearn.preprocessing import LabelEncoder

# Assuming your training labels are stored in a NumPy array called train_labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
num_classes = len(label_encoder.classes_)
print("Number of unique labels:", num_classes)

# test_labels_encoded = label_encoder.transform(test_labels)

# save the label encoder
import pickle
with open('label_encoder.pkl', 'wb') as le_dump_file:
    pickle.dump(label_encoder, le_dump_file)
    



Number of unique labels: 1301


In [23]:
# decode the labels from the stored encoder
import pickle
with open('label_encoder.pkl', 'rb') as le_load_file:
    label_encoder = pickle.load(le_load_file)

label_encoder.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

array([ 0,  1,  4,  6,  7,  8, 10, 12, 13, 16])

In [ ]:
# save the train and test labels and images
np.save('train_images.npy', train_images)
np.save('train_labels.npy', train_labels_encoded)
np.save('test_images.npy', test_images)
np.save('test_labels.npy', test_labels)


In [6]:
# code to load the saved files
import numpy as np
train_images = np.load('/content/drive/MyDrive/deepface/train_images.npy')
train_labels_encoded = np.load('/content/drive/MyDrive/deepface/train_labels.npy')
test_images = np.load('/content/drive/MyDrive/deepface/test_images.npy')
test_labels = np.load('/content/drive/MyDrive/deepface/test_labels.npy')


In [10]:
num_classes=1301

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Load the pre-trained FaceNet model
facenet_model = Face512

# Freeze the face512 layers
for layer in facenet_model.layers[:-3]:
    layer.trainable = False

# Add a new dense layer for classification
inputs = Input(shape=(160, 160, 3))
fco = facenet_model(inputs)
# x = Dense(700, activation='relu')(x)
x = Dense(128, activation='relu')(fco)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
# model.compile(optimizer=Adam(lr=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# checkpoint = ModelCheckpoint('model.h5', monitor='accuracy', save_best_only=True)
# model.fit(train_images, train_labels_encoded, epochs=100, callbacks=[checkpoint])

# # Unfreeze the top layers of the FaceNet model
# for layer in facenet_model.layers[-10:]:
#     layer.trainable = True

# # Compile the model again
# model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the model again
# model.fit(train_images, train_labels_encoded, epochs=10, callbacks=[checkpoint])

In [10]:
for layer in facenet_model.layers[-15:]:
    layer.trainable = True

# Compile the model again
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model again
model.fit(train_images, train_labels_encoded, epochs=40, callbacks=[checkpoint])

Epoch 1/40
154/154 [==============================] - 16s 44ms/step - loss: 2.3681 - accuracy: 0.5669
Epoch 2/40
154/154 [==============================] - 8s 49ms/step - loss: 0.8197 - accuracy: 0.7808
Epoch 3/40
154/154 [==============================] - 7s 44ms/step - loss: 0.6065 - accuracy: 0.8353
Epoch 4/40
154/154 [==============================] - 7s 46ms/step - loss: 0.5446 - accuracy: 0.8465
Epoch 5/40
154/154 [==============================] - 7s 44ms/step - loss: 0.4466 - accuracy: 0.8724
Epoch 6/40
154/154 [==============================] - 7s 46ms/step - loss: 0.4339 - accuracy: 0.8730
Epoch 7/40
154/154 [==============================] - 7s 43ms/step - loss: 0.4318 - accuracy: 0.8752
Epoch 8/40
154/154 [==============================] - 7s 45ms/step - loss: 0.4289 - accuracy: 0.8760
Epoch 9/40
154/154 [==============================] - 7s 43ms/step - loss: 0.3085 - accuracy: 0.9060
Epoch 10/40
154/154 [==============================] - 7s 45ms/step - loss: 0.3378 - accur

In [11]:
model.save('model1.h5')

In [3]:
!cp /content/drive/MyDrive/deepface/model1.h5 /content/

In [4]:
from tensorflow.keras.models import load_model
model = load_model('model1.h5')

/usr/local/lib/python3.10/dist-packages/keras/layers/core/lambda_layer.py:325: UserWarning: deepface.basemodels.Facenet is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


In [5]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 inception_resnet_v1 (Functi  (None, 512)              23497424  
 onal)                                                           
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dense_6 (Dense)             (None, 1301)              167829    
                                                                 
Total params: 23,730,917
Trainable params: 2,296,469
Non-trainable params: 21,434,448
_________________________________________________________________


In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', patience=10, verbose=1, mode='max', restore_best_weights=True)
facenet_model = Face512

for layer in facenet_model.layers[-30:]:
    layer.trainable = True
checkpoint = ModelCheckpoint('model.h5', monitor='accuracy', save_best_only=True)
# Compile the model again
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model again
model.fit(train_images, train_labels_encoded, epochs=40, callbacks=[checkpoint, early_stopping])

Epoch 1/40
154/154 [==============================] - 29s 57ms/step - loss: 0.2334 - accuracy: 0.9334
Epoch 2/40
154/154 [==============================] - 8s 51ms/step - loss: 0.2141 - accuracy: 0.9381
Epoch 3/40
154/154 [==============================] - 7s 43ms/step - loss: 0.2290 - accuracy: 0.9353
Epoch 4/40
154/154 [==============================] - 7s 45ms/step - loss: 0.2082 - accuracy: 0.9355
Epoch 5/40
154/154 [==============================] - 8s 49ms/step - loss: 0.1755 - accuracy: 0.9448
Epoch 6/40
154/154 [==============================] - 8s 51ms/step - loss: 0.1771 - accuracy: 0.9507
Epoch 7/40
154/154 [==============================] - 7s 44ms/step - loss: 0.1911 - accuracy: 0.9481
Epoch 8/40
154/154 [==============================] - 7s 46ms/step - loss: 0.1682 - accuracy: 0.9495
Epoch 9/40
154/154 [==============================] - 8s 49ms/step - loss: 0.1663 - accuracy: 0.9540
Epoch 10/40
154/154 [==============================] - 7s 45ms/step - loss: 0.1549 - accur

In [16]:
for layer in facenet_model.layers[-60:]:
    layer.trainable = True
checkpoint = ModelCheckpoint('model.h5', monitor='accuracy', save_best_only=True)
# Compile the model again
model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model again
model.fit(train_images, train_labels_encoded, epochs=50, callbacks=[checkpoint, early_stopping])

Epoch 1/50
154/154 [==============================] - 16s 52ms/step - loss: 0.1358 - accuracy: 0.9587
Epoch 2/50
154/154 [==============================] - 8s 49ms/step - loss: 0.1588 - accuracy: 0.9607
Epoch 3/50
154/154 [==============================] - 7s 46ms/step - loss: 0.1629 - accuracy: 0.9571
Epoch 4/50
154/154 [==============================] - 8s 49ms/step - loss: 0.1227 - accuracy: 0.9642
Epoch 5/50
154/154 [==============================] - 7s 46ms/step - loss: 0.1515 - accuracy: 0.9546
Epoch 6/50
154/154 [==============================] - 8s 49ms/step - loss: 0.1203 - accuracy: 0.9662
Epoch 7/50
154/154 [==============================] - 7s 45ms/step - loss: 0.1139 - accuracy: 0.9660
Epoch 8/50
154/154 [==============================] - 7s 43ms/step - loss: 0.1385 - accuracy: 0.9619
Epoch 9/50
154/154 [==============================] - 7s 45ms/step - loss: 0.1650 - accuracy: 0.9532
Epoch 10/50
154/154 [==============================] - 7s 43ms/step - loss: 0.1053 - accur

In [17]:
for layer in facenet_model.layers[-100:]:
    layer.trainable = True
checkpoint = ModelCheckpoint('model.h5', monitor='accuracy', save_best_only=True)
# Compile the model again
model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model again
model.fit(train_images, train_labels_encoded, epochs=50, callbacks=[checkpoint, early_stopping])

Epoch 1/50
154/154 [==============================] - 16s 54ms/step - loss: 0.1053 - accuracy: 0.9719
Epoch 2/50
154/154 [==============================] - 7s 44ms/step - loss: 0.1084 - accuracy: 0.9709
Epoch 3/50
154/154 [==============================] - 8s 51ms/step - loss: 0.0926 - accuracy: 0.9725
Epoch 4/50
154/154 [==============================] - 7s 44ms/step - loss: 0.1681 - accuracy: 0.9560
Epoch 5/50
154/154 [==============================] - 7s 46ms/step - loss: 0.1684 - accuracy: 0.9581
Epoch 6/50
154/154 [==============================] - 7s 44ms/step - loss: 0.1472 - accuracy: 0.9599
Epoch 7/50
154/154 [==============================] - 7s 46ms/step - loss: 0.1212 - accuracy: 0.9660
Epoch 8/50
154/154 [==============================] - 7s 43ms/step - loss: 0.1159 - accuracy: 0.9689
Epoch 9/50
154/154 [==============================] - 8s 50ms/step - loss: 0.0913 - accuracy: 0.9752
Epoch 10/50
154/154 [==============================] - 8s 49ms/step - loss: 0.0842 - accur

In [18]:
# Freeze the model
!pip install tensorflowjs
model.trainable = False

# Convert the model to TFJS
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'tfjs_model')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: flax
    Found existing installation: flax 0.6.10
    Uninstalling flax-0.6.10:
      Successfully uninstalled flax-0.6.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.5 requires packaging>=21.3, but you have packaging 20.9 which is incompatible.


In [20]:
!cp -r /content/tfjs_model /content/drive/MyDrive/deepface/

In [21]:
model.save('model1.h5')

In [22]:
!cp /content/model1.h5 /content/drive/MyDrive/deepface/

In [2]:
from tensorflow.keras.models import load_model

model = load_model('model1.h5', compile=False)


In [3]:
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 inception_resnet_v1 (Functi  (None, 512)              23497424  
 onal)                                                           
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dense_6 (Dense)             (None, 1301)              167829    
                                                                 
Total params: 23,730,917
Trainable params: 0
Non-trainable params: 23,730,917
_________________________________________________________________


In [12]:
from tensorflow.keras.models import Model

FeatureExtractor = Model(inputs=model.input, outputs=fco)

FeatureExtractor.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 inception_resnet_v1 (Functi  (None, 512)              23497424  
 onal)                                                           
                                                                 
Total params: 23,497,424
Trainable params: 918,016
Non-trainable params: 22,579,408
_________________________________________________________________


In [13]:
Classifier = Model(inputs=fco, outputs=outputs)

Classifier.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512)]             0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 1301)              167829    
                                                                 
Total params: 233,493
Trainable params: 233,493
Non-trainable params: 0
_________________________________________________________________


In [14]:
FeatureExtractor.save('FeatureExtractor.h5')    

Classifier.save('Classifier.h5') 

 